In [16]:
import numpy as np

# Load dataset randomly with 6 features
def gen_dataset(length) :
    dataset = np.empty((length, 8))
    for i in xrange(length) :
        while True :
            dataset[i][0] = i
            dataset[i][1] = np.random.randint(0, 101)
            dataset[i][2] = np.random.randint(0, 101)
            dataset[i][3] = np.random.randint(0, 101)
            dataset[i][4] = np.random.randint(0, 101)
            dataset[i][5] = np.random.randint(0, 101)
            dataset[i][6] = np.random.randint(0, 101)
            dataset[i][7] = np.random.randint(0, 2)
            
            for j in xrange(i) :
                if (dataset[j][1] == dataset[i][1] and
                    dataset[j][2] == dataset[i][2] and
                    dataset[j][3] == dataset[i][3] and
                    dataset[j][4] == dataset[i][4] and
                    dataset[j][5] == dataset[i][5] and
                    dataset[j][6] == dataset[i][6]) :
                        break
            else :
                break
    
    # Randomly permutate the dataset
    np.random.shuffle(dataset)
    # Split dataset into train and test set
    split_index = 4*length/5
    train, test = dataset[:split_index, :], dataset[split_index:, :]
    # Save them to csv files
    np.savetxt('train.csv', train, delimiter=',')
    np.savetxt('test.csv', test, delimiter=',')

In [17]:
import numpy as np

# Read train and test dataset from train and test files
def read_dataset() :
    train = np.genfromtxt('train.csv', delimiter=',')
    test = np.genfromtxt('test.csv', delimiter=',')
    train_x = train[:, 1:7]
    train_y = train[:, 7]
    test_x = test[:, 1:7]
    test_y = test[:, 7]
    return train_x, train_y, test_x, test_y

In [18]:
# KNN from sklean library, train and predict
def sklearn_knn(n, train_x, train_y, test_x) :
    from sklearn.neighbors import KNeighborsClassifier
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_x, train_y)
    return neigh.predict(test_x)

In [19]:
import numpy as np

# Implement knn with Euclidean distance
def run_knn(n, train_x, train_y, test_x) :
    no_test = len(test_x)
    test_predict = np.empty(no_test)
    # For all points in test set
    for i in xrange(no_test) :
        distances = np.empty((len(train_x), 2))
        # Calcuate distances and outputs from other train points
        for j in xrange(len(train_x)) :
            distance = 0
            for k in xrange(6) :
                distance += (test_x[i][k]-train_x[j][k])**2
            distances[j][0] = distance
            distances[j][1] = train_y[j]
        # Sort the outputs according to distances
        distances = distances[np.argsort(distances[:, 0])]
        votes = [0, 0]
        # For nearest k points calculate votes for class 0 and 1
        for k in xrange(n) :
            if distances[k][1] == 0 :
                votes[0] += 1
            else :
                votes[1] += 1
        # If votes for class 0 is higher then predict 0, else if
        # for class 1 is higher then predict 1 else predict randomly
        if votes[0] > votes[1] :
            test_predict[i] = 0
        elif votes[0] < votes[1] :
            test_predict[i] = 1
        else :
            test_predict[i] = np.random.randint(0, 2)
    # Return predicted output
    return test_predict

In [20]:
# Initalize dataset
gen_dataset(1000)
train_x, train_y, test_x, test_y = read_dataset()

# Calculate accuracy of the predicted data
def get_accuracy(test_result_y, test_y):
    correct = 0;
    for i in xrange(len(test_y)):
        if test_y[i] == test_result_y[i]:
            correct += 1
    return correct, float(correct)/len(test_y)

In [25]:
import time

# Run Knn for k = 1 to 21
print "| K | My KNN | Sklearn KNN | My KNN Time | Sklearn Time |"
print "| --- | --- | --- | --- | --- |"
for k in xrange(21) :
    t1 = time.time()
    predicted_y_knn = run_knn(k+1, train_x, train_y, test_x)
    t2 = time.time()
    predicted_y_skl = sklearn_knn(k+1, train_x, train_y, test_x)
    t3 = time.time()

    _, accuracy_knn = get_accuracy(predicted_y_knn, test_y)
    _, accuracy_skl = get_accuracy(predicted_y_skl, test_y)

    print "|", k+1, "|", accuracy_knn, "|", accuracy_skl, "|", t2-t1, "|", t3-t2, "|"
#     print "Accuracy for k =", k+1, "using own knn is", accuracy_knn
#     print "Accuracy for k =", k+1, "using sklearn knn is", accuracy_skl

| K | My KNN | Sklearn KNN | My KNN Time | Sklearn Time |
| --- | --- | --- | --- | --- |
| 1 | 0.47 | 0.47 | 1.06377601624 | 0.00253796577454 |
| 2 | 0.465 | 0.445 | 1.11978387833 | 0.00261116027832 |
| 3 | 0.51 | 0.51 | 1.0166490078 | 0.00270795822144 |
| 4 | 0.485 | 0.48 | 1.02640390396 | 0.00284910202026 |
| 5 | 0.485 | 0.485 | 1.05628204346 | 0.00295209884644 |
| 6 | 0.485 | 0.505 | 1.07843494415 | 0.00317692756653 |
| 7 | 0.475 | 0.475 | 1.07187008858 | 0.00445199012756 |
| 8 | 0.46 | 0.445 | 1.04060506821 | 0.00319004058838 |
| 9 | 0.485 | 0.485 | 1.19458293915 | 0.00343108177185 |
| 10 | 0.5 | 0.46 | 1.04880094528 | 0.00426316261292 |
| 11 | 0.52 | 0.52 | 1.09572482109 | 0.00343203544617 |
| 12 | 0.495 | 0.485 | 1.07432603836 | 0.00353598594666 |
| 13 | 0.525 | 0.52 | 1.05930709839 | 0.00361180305481 |
| 14 | 0.535 | 0.49 | 1.10788202286 | 0.00373101234436 |
| 15 | 0.52 | 0.52 | 1.13192391396 | 0.00362920761108 |
| 16 | 0.52 | 0.51 | 1.05603289604 | 0.00387096405029 |
| 17 | 0.

## Report

* I ran my own implementatation of knn and sklearn knn and compared their accuracy on their outputs.
* The Accuracies for k = 1 to 21 are :

| K | My KNN | Sklearn KNN | My KNN Time | Sklearn Time |
| --- | --- | --- | --- | --- |
| 1 | 0.47 | 0.47 | 1.06377601624 | 0.00253796577454 |
| 2 | 0.465 | 0.445 | 1.11978387833 | 0.00261116027832 |
| 3 | 0.51 | 0.51 | 1.0166490078 | 0.00270795822144 |
| 4 | 0.485 | 0.48 | 1.02640390396 | 0.00284910202026 |
| 5 | 0.485 | 0.485 | 1.05628204346 | 0.00295209884644 |
| 6 | 0.485 | 0.505 | 1.07843494415 | 0.00317692756653 |
| 7 | 0.475 | 0.475 | 1.07187008858 | 0.00445199012756 |
| 8 | 0.46 | 0.445 | 1.04060506821 | 0.00319004058838 |
| 9 | 0.485 | 0.485 | 1.19458293915 | 0.00343108177185 |
| 10 | 0.5 | 0.46 | 1.04880094528 | 0.00426316261292 |
| 11 | 0.52 | 0.52 | 1.09572482109 | 0.00343203544617 |
| 12 | 0.495 | 0.485 | 1.07432603836 | 0.00353598594666 |
| 13 | 0.525 | 0.52 | 1.05930709839 | 0.00361180305481 |
| 14 | 0.535 | 0.49 | 1.10788202286 | 0.00373101234436 |
| 15 | 0.52 | 0.52 | 1.13192391396 | 0.00362920761108 |
| 16 | 0.52 | 0.51 | 1.05603289604 | 0.00387096405029 |
| 17 | 0.53 | 0.53 | 1.13083600998 | 0.00407695770264 |
| 18 | 0.52 | 0.525 | 1.0790951252 | 0.00394082069397 |
| 19 | 0.525 | 0.525 | 1.10567522049 | 0.00395393371582 |
| 20 | 0.515 | 0.495 | 1.08136296272 | 0.00401210784912 |
| 21 | 0.515 | 0.515 | 1.13130116463 | 0.00410103797913 |

* For odd k both my implementation and sklean predicted with same accuracy
* For even the accuracies are differnet and in case of tie I give random prediction
* In case of time, mine took larger time than that of sklearn as it is optimised
* There was a increase in time from k = 1 to 21 although not so steadily with many ups and downs